# TP-MCTS on Colab

This notebook clones the repo and runs a single experiment.

In [2]:
# Clone repo
%cd /content
!rm -rf tp_mcts
!git clone https://github.com/eliezerRevach/tp_mcts.git
%cd /content/tp_mcts

/content
Cloning into 'tp_mcts'...
remote: Enumerating objects: 2296, done.
remote: Counting objects: 100% (441/441), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 2296 (delta 211), reused 385 (delta 167), pack-reused 1855 (from 1)
Receiving objects: 100% (2296/2296), 15.35 MiB | 17.22 MiB/s, done.
Resolving deltas: 100% (1373/1373), done.
/content/tp_mcts


In [3]:
%%bash
set -e
cd /content/tp_mcts

echo "Repo commit:"
git rev-parse --short HEAD
git log -1 --oneline

echo
python - <<'PY'
import unified_planning
import unified_planning.parser as p

print("unified_planning package:", unified_planning.__file__)
print("parser file:", p.__file__)
with open(p.__file__, "r") as f:
    text = f.read()
print("--reward_mode present:", "reward_mode" in text)
PY

Repo commit:
3a69e8f
3a69e8f Adds goal reward control and CRN heuristic

unified_planning package: /content/tp_mcts/unified_planning/__init__.py
parser file: /content/tp_mcts/unified_planning/parser.py
--reward_mode present: True


In [4]:
# Install dependencies
!pip -q install dill numpy

In [5]:
# Global run config (single Rover(1) run, rootInterval only)
# HEURISTIC_MODE "legacy": each node runs its own heuristic (single TRPG per node, no CRN).
CONFIG = {
    "DOMAIN": "nasa_rover",
    "OBJECT_AMOUNT": 1,
    "DEADLINE": 20,
    "RUNS": 100,
    "SEARCH_TIME": 10,
    "SEARCH_DEPTH": 40,
    "EXPLORATION_CONSTANT": 10,
    "HEURISTIC_SAMPLES": 1,
    "HEURISTIC_MODE": "legacy",
    "SELECTION_TYPE": "rootInterval",
    "REWARD_MODE": "deadline",
    "SEED": 123,
}

import json
config_path = "/content/tp_mcts/run_config.json"
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(CONFIG, f)

print("Wrote", config_path)
print(CONFIG)

Wrote /content/tp_mcts/run_config.json
{'DOMAIN': 'nasa_rover', 'OBJECT_AMOUNT': 1, 'DEADLINE': 20, 'RUNS': 100, 'SEARCH_TIME': 10, 'SEARCH_DEPTH': 40, 'EXPLORATION_CONSTANT': 10, 'HEURISTIC_SAMPLES': 1, 'SELECTION_TYPE': 'rootInterval', 'REWARD_MODE': 'deadline', 'SEED': 123}


In [ ]:
%%bash
# Single Rover(1) run (rootInterval only)
set -e
cd /content/tp_mcts

python - <<'PY'
import json
import subprocess
import shlex
import time

with open('/content/tp_mcts/run_config.json', 'r', encoding='utf-8') as f:
    cfg = json.load(f)

tag = f"rover1_{cfg['SELECTION_TYPE']}_{cfg['SEARCH_TIME']}s"
log = f"run_{tag}.log"
cmd = (
    "python -u unified_planning/run_domain.py "
    f"--domain {cfg['DOMAIN']} "
    f"--object_amount {cfg['OBJECT_AMOUNT']} "
    f"--deadline {cfg['DEADLINE']} "
    f"--runs {cfg['RUNS']} "
    f"--search_time {cfg['SEARCH_TIME']} "
    f"--search_depth {cfg['SEARCH_DEPTH']} "
    f"--exploration_constant {cfg['EXPLORATION_CONSTANT']} "
    f"--selection_type {cfg['SELECTION_TYPE']} "
    f"--heuristic_samples {cfg['HEURISTIC_SAMPLES']} "
    f"--heuristic_mode {cfg.get('HEURISTIC_MODE', 'legacy')} "
    f"--reward_mode {cfg['REWARD_MODE']} "
    f"--seed {cfg['SEED']}"
)

print(f"\n=== Running {tag} ===")
print(cmd)
with open(log, 'w', encoding='utf-8') as out:
    p = subprocess.Popen(shlex.split(cmd), stdout=out, stderr=subprocess.STDOUT)

while p.poll() is None:
    print(f"[{tag}] still running...")
    try:
        with open(log, 'r', encoding='utf-8') as lf:
            lines = lf.readlines()
        interesting = [ln.strip() for ln in lines if (
            ln.startswith('Started round') or
            ln.startswith('started step') or
            ln.startswith('Completed') or
            ln.startswith('Amount of success') or
            ln.startswith('Average success time') or
            ln.startswith('STD success time')
        )]
        for ln in interesting[-4:]:
            print('  ', ln)
    except FileNotFoundError:
        pass
    time.sleep(30)

if p.returncode != 0:
    raise RuntimeError(f"Run failed for {tag} (exit={p.returncode})")

print(f"--- summary {tag} ---")
with open(log, 'r', encoding='utf-8') as lf:
    for ln in lf:
        if ln.startswith('Completed') or ln.startswith('Amount of success') or ln.startswith('Average success time') or ln.startswith('STD success time'):
            print(ln.strip())
PY

In [ ]:
%%bash
# Quick summary for the single run
set -e
cd /content/tp_mcts

python - <<'PY'
import json
with open('/content/tp_mcts/run_config.json', 'r', encoding='utf-8') as f:
    cfg = json.load(f)
log = f"run_rover1_{cfg['SELECTION_TYPE']}_{cfg['SEARCH_TIME']}s.log"
print(f"=== {log} ===")
print('If file exists, summary will be shown below.')
PY

grep -E "Completed|Amount of success|Average success time|STD success time" run_rover1_*.log || true

Process is interrupted.


In [ ]:
%%bash
# Optional compact printout for this single rootInterval run
set -e
cd /content/tp_mcts

python - <<'PY'
import json
with open('/content/tp_mcts/run_config.json', 'r', encoding='utf-8') as f:
    cfg = json.load(f)
print(f"selection={cfg['SELECTION_TYPE']}, budget={cfg['SEARCH_TIME']}s")
PY

grep -E "Amount of success|Average success time|STD success time" run_rover1_*.log || true

=== reward_mode=terminal ===
Completed = 40
Amount of success = 40
Average success time = 4
STD success time = 0.0
=== reward_mode=deadline ===
Completed = 40
Amount of success = 40
Average success time = 4
STD success time = 0.0


In [ ]:
import sys, platform, os
print("executable:", sys.executable)
print("platform:", platform.platform())
print("cwd:", os.getcwd())
print("has /content:", os.path.exists("/content"))

executable: /usr/bin/python3
platform: Linux-6.6.105+-x86_64-with-glibc2.35
cwd: /content/tp_mcts
has /content: True


In [ ]:
!lscpu


Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             48 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      24
  On-line CPU(s) list:       0-23
Vendor ID:                   AuthenticAMD
  Model name:                AMD EPYC 7B13
    CPU family:              25
    Model:                   1
    Thread(s) per core:      2
    Core(s) per socket:      12
    Socket(s):               1
    Stepping:                0
    BogoMIPS:                4899.99
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ht s
                             yscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constan
                             t_tsc rep_good nopl nonstop_tsc cpuid extd_apicid t
                             sc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse
                             4_1 